# EXEMPLE DE SCRAPPING

### MONTOYA ADELE

ESILV Année 5 - DIA02

### SUJET ET CONTEXTE : 

Pour rappel notre projet :  
Etude comparative entre les restaurants qui respectent la saisonnalité des fruits et légumes et ceux qui ne respectent pas.

Données :  
Pour ce projet, en plus de scrapper les menus des restaurants, il faut scrapper la liste des fruits et légumes pour chaque saison. Dans notre cas, on va scrapper la liste des aliments en fonction des mois.

SiteWeb :  
J'ai trouvé ces informations sur le site https://www.greenpeace.fr/guetteur/calendrier/

Prérequis :  
Avoir Selenium, et avoir des versions compatibles de Goolgle Chrome et Chrome Driver (ce qui pour ma part était compliqué à avoir)

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup


# Configurer les options de Selenium pour exécuter Chrome en mode headless
options = Options()
options.add_argument('--headless')

path_to_chromedriver = 'D:\ESILV4\Python data analyse\chromedriver-win32\chromedriver.exe' 
driver = webdriver.Chrome(executable_path=path_to_chromedriver)
driver.get("https://www.google.com")

url = 'https://www.greenpeace.fr/guetteur/calendrier/' #mon site à scrapper

driver.get(url) # Ouvrir la page web avec Selenium

# Attendre que le contenu dynamique soit chargé
wait = WebDriverWait(driver, 10)
wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "month")))

# stocker le code source de la page dans la variable htlm
html = driver.page_source

# Fermer le navigateur
driver.quit()

# Utiliser BeautifulSoup pour analyser le contenu
soup = BeautifulSoup(html, 'html.parser')

mois_elements = soup.find_all('li', class_='month')

for mois in mois_elements:
    nom_du_mois = mois.find('a').get('id') # Récupérer les noms des mois

    # Le premier 'ul' pour les légumes
    ul_legumes = mois.find_next('ul', attrs={'data-v-a603cee8': True})
    legumes = [li.get_text(strip=True) for li in ul_legumes.find_all('li', attrs={'data-v-a603cee8': True})]

    #  Après celui des légumes, c'est le ul des fruits
    ul_fruits = ul_legumes.find_next('ul', attrs={'data-v-a603cee8': True})
    fruits = []
    if ul_fruits:
        fruits = [li.get_text(strip=True) for li in ul_fruits.find_all('li', attrs={'data-v-a603cee8': True})]
        
    # Et le dernier c'est celui des Céréales & légumineuses
    ul_cere = ul_fruits.find_next('ul', attrs={'data-v-a603cee8': True})
    cereales = []
    if ul_cere:
        cereales = [li.get_text(strip=True) for li in ul_cere.find_all('li', attrs={'data-v-a603cee8': True})]

    # j'ajoute dans le dico avec le nom du mois comme clé
    noms_par_mois[nom_du_mois] = {'Legumes': legumes, 'Fruits': fruits, 'Céréales et légumineuses': cereales}

#affichage
for mois, categories in noms_par_mois.items():
    print(f"En {mois.capitalize()} :")
    for categorie, items in categories.items():
        items_list = ', '.join(items)
        print(f"{categorie} : {items_list}")
    print()

En Janvier :
Legumes : Ail, Betterave, Carotte, Céleri-branche, Céleri-rave, Chou, Chou blanc, Chou de Bruxelles, Chou frisé, Chou rouge, Courge, Crosne, Endive, Épinard, Frisée, Mâche, Navet, Oignon, Panais, Poireau, Pomme de terre de conservation, Salsifi, Topinambour
Fruits : Amande sèche, Citron, Clémentine, Kaki, Kiwi, Mandarine, Orange, Pamplemousse, Physalis, Poire, Pomme
Céréales et légumineuses : Lentille

En Fevrier :
Legumes : Ail, Betterave, Carotte, Céleri-rave, Chou blanc, Chou de Bruxelles, Chou frisé, Chou rouge, Crosne, Endive, Épinard, Frisée, Mâche, Navet, Oignon, Panais, Poireau, Pomme de terre de conservation, Radis, Salsifi, Topinambour
Fruits : Amande sèche, Citron, Clémentine, Kiwi, Mandarine, Orange, Pamplemousse, Physalis, Poire, Pomme
Céréales et légumineuses : Lentille

En Mars :
Legumes : Ail, Asperge, Betterave, Blette, Carotte, Céleri-rave, Chou, Chou de Bruxelles, Chou-fleur, Crosne, Endive, Épinard, Frisée, Navet, Oignon, Panais, Poireau, Pomme de terre

In [4]:
import pandas as pd

data = []

for mois, categories in noms_par_mois.items():
    for categorie, elements in categories.items():
        data.append([mois, categorie, ', '.join(elements)])

df = pd.DataFrame(data, columns=['Mois', 'Catégorie', 'Liste_elements'])

print(df)

         Mois                 Catégorie  \
0     janvier                   Legumes   
1     janvier                    Fruits   
2     janvier  Céréales et légumineuses   
3     fevrier                   Legumes   
4     fevrier                    Fruits   
5     fevrier  Céréales et légumineuses   
6        mars                   Legumes   
7        mars                    Fruits   
8        mars  Céréales et légumineuses   
9       avril                   Legumes   
10      avril                    Fruits   
11      avril  Céréales et légumineuses   
12        mai                   Legumes   
13        mai                    Fruits   
14        mai  Céréales et légumineuses   
15       juin                   Legumes   
16       juin                    Fruits   
17       juin  Céréales et légumineuses   
18    juillet                   Legumes   
19    juillet                    Fruits   
20    juillet  Céréales et légumineuses   
21       aout                   Legumes   
22       ao

In [6]:
# Enregistrer le df en .xlsx
chemin_fichier_excel = 'D:\\ESILV5\\Webscrapping\\Exemple de scrapp\\liste_aliments_mois.xlsx'
df.to_excel(chemin_fichier_excel, index=False)